In [1]:
import datetime

import pandas as pd
import numpy as np

import sqlite3
import os

import sys

sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

sys.path.append("../")
from utils import SqliteUtils

In [143]:
length = 3
people = 2

db_local_path = 'db.sqlite3'
sql = SqliteUtils(db_local_path)

# Select recipes

sql_query = """
        select rec.recipe_name, 
               rec.url,
               facts.ingredient,
               facts.unit_measure,
               facts.quantity
        from weekly_planner_recipe rec
        left join weekly_planner_recipe_fact facts
        using(recipe_name)
"""

recipes = sql.sqlite_to_pandas(sql_query)
rand_recipe_names = np.random.choice(recipes['recipe_name'].unique(),
                                size=length, 
                                replace=False)

rand_recipes = recipes[recipes['recipe_name'].isin(rand_recipe_names)].copy()
rand_recipes['quantity'] = rand_recipes['quantity'].astype(float)
rand_recipes_agg = rand_recipes.groupby('recipe_name').agg({'ingredient': list,
                                          'url': max
                                        }).reset_index()

# Fetch Ingredients
recipe_dict = {}
ingredient_tuples = []

for rr in rand_recipes_agg.itertuples():

    recipe_dict[rr.recipe_name] = {'ingredients_list': ", ".join([x for x in rr.ingredient]),
                                   'ingredients_tuples': tuple(rr.ingredient),
                                   'url': rr.url}
    
ingredient_tuples = rand_recipes. \
              groupby(['ingredient', 'unit_measure']). \
              agg({'quantity':np.sum}). \
              reset_index()

# Create Shopping list
sl = ShoppingList(ingredient_tuples, people)
shopping_list = sl.create_shopping_list()

In [144]:
recipe_dict

{'Insalata di mare': {'ingredients_list': "Aglio, Alloro, Calamari, Carote, Cozze, Gamberi, Olio extravergine d'oliva, Pepe nero, Pepe rosa in grani, Polpo, Prezzemolo, Sale fino, Sedano, Succo di limone, Vongole",
  'ingredients_tuples': ('Aglio',
   'Alloro',
   'Calamari',
   'Carote',
   'Cozze',
   'Gamberi',
   "Olio extravergine d'oliva",
   'Pepe nero',
   'Pepe rosa in grani',
   'Polpo',
   'Prezzemolo',
   'Sale fino',
   'Sedano',
   'Succo di limone',
   'Vongole'),
  'url': 'https://ricette.giallozafferano.it/Insalata-di-mare.html'},
 'Tasche di pollo ripiene di gamberetti': {'ingredients_list': "Acqua, Aglio, Burro, Carote, Gamberetti, Gamberetti, Noce moscata, Olio extravergine d'oliva, Olio extravergine d'oliva, Pane, Pepe nero, Petto di pollo, Prezzemolo, Prezzemolo, Sale, Sale fino, Scalogno, Sedano, Timo, Vino bianco, Zafferano",
  'ingredients_tuples': ('Acqua',
   'Aglio',
   'Burro',
   'Carote',
   'Gamberetti',
   'Gamberetti',
   'Noce moscata',
   "Olio extra

In [133]:
# Create Shopping list
sl = ShoppingList(ingredient_tuples, people)
shopping_list = sl.create_shopping_list()

shopping_list

{'Aceto di vino rosso': {'unit_measure': 'ml', 'quantity': '100'},
 'Acqua': {'unit_measure': 'ml', 'quantity': '650'},
 'Aglio': {'unit_measure': 'gr', 'quantity': '20'},
 'Amido di mais': {'unit_measure': 'gr', 'quantity': '20'},
 'Burro': {'unit_measure': 'gr', 'quantity': '40'},
 'Controfiletto di manzo': {'unit_measure': 'kg', 'quantity': '500'},
 'Filetto di merluzzo': {'unit_measure': 'gr', 'quantity': '300'},
 'Fiocchi di sale': {'unit_measure': 'q.b.', 'quantity': ' '},
 'Limoni': {'unit_measure': 'gr', 'quantity': '100'},
 'Olio di semi di arachide': {'unit_measure': 'ml', 'quantity': '30'},
 "Olio extravergine d'oliva": {'unit_measure': 'ml', 'quantity': '80'},
 'Orata': {'unit_measure': 'gr', 'quantity': '900'},
 'Origano': {'unit_measure': 'gr', 'quantity': '10'},
 'Patate': {'unit_measure': 'gr', 'quantity': '700'},
 'Pepe nero': {'unit_measure': 'q.b.', 'quantity': ' '},
 'Prezzemolo': {'unit_measure': 'gr', 'quantity': '80'},
 'Prosciutto crudo': {'unit_measure': 'gr', 

In [132]:
class ShoppingList(object):

    def __init__(self, ingredient_tuples, people):

        self.ingredient_tuples = ingredient_tuples
        self.people = people

    def create_shopping_list(self):

        # Group by ingredient and unit_measure
        ing_df = ingredient_tuples.groupby(['ingredient', 'unit_measure'], as_index=False).sum()
        ing_df = ing_df.sort_values(by = 'ingredient')
        ing_df = ing_df.set_index('ingredient')

        # Multiply quantities by number of people
        ing_df['quantity'] = ing_df['quantity'] * self.people

        # Round to nearest 10'
        ing_df['quantity'] = [x + (10-(x % 10)) 
                              if x % 10 > 0 
                              else x 
                              for x in ing_df['quantity']]

        # Convert to str and replace 0 values with empty string
        ing_df['quantity'] = ing_df['quantity'].astype('int').astype('str')
        ing_df['quantity'] = ing_df['quantity'].replace('0', ' ')

        # Replace nas in unite unit measure with empty string
        ing_df['unit_measure'] = ing_df['unit_measure'].astype('str')
        ing_df['unit_measure'] = ing_df['unit_measure'].replace('nan', ' ')

        # Create ingredients list
        shopping_list = ing_df.transpose().to_dict()

        return shopping_list

In [135]:
def planner(request):

    length = int(request.GET.get('recipes', 3))
    people = int(request.GET.get('people', 2))

    db_local_path = 'db.sqlite3'
sql = SqliteUtils(db_local_path)

# Select recipes

sql_query = """
        select rec.recipe_name, 
               rec.url,
               facts.ingredient,
               facts.unit_measure,
               facts.quantity
        from weekly_planner_recipe rec
        left join weekly_planner_recipe_fact facts
        using(recipe_name)
"""

recipes = sql.sqlite_to_pandas(sql_query)
rand_recipe_names = np.random.choice(recipes['recipe_name'].unique(),
                                size=length, 
                                replace=False)

rand_recipes = recipes[recipes['recipe_name'].isin(rand_recipe_names)].copy()
rand_recipes['quantity'] = rand_recipes['quantity'].astype(float)
rand_recipes_agg = rand_recipes.groupby('recipe_name').agg({'ingredient': list,
                                          'url': max
                                        }).reset_index()

# Fetch Ingredients
recipe_dict = {}
ingredient_tuples = []

for rr in rand_recipes_agg.itertuples():

    recipe_dict[rr.recipe_name] = {'ingredients_list': rr.ingredient,
                                   'ingredients_tuples': tuple(rr.ingredient),
                                   'url': rr.url}
    
ingredient_tuples = rand_recipes. \
              groupby(['ingredient', 'unit_measure']). \
              agg({'quantity':np.sum}). \
              reset_index()

# Create Shopping list
sl = ShoppingList(ingredient_tuples, people)
shopping_list = sl.create_shopping_list()

    return render(request, 'weekly_planner/planner.html', {'recipes':recipe_dict, 'shopping_list': shopping_list})



class ShoppingList(object):

    def __init__(self, ingredient_tuples, people):

        self.ingredient_tuples = ingredient_tuples
        self.people = people

    def create_shopping_list(self):

        # Create Ingredients Pandas DataFrame
        vars = ['ingredient', 'quantity', 'unit_measure']
        ing_df = pd.DataFrame(self.ingredient_tuples, columns=vars)
        ing_df['quantity'] = ing_df['quantity'].astype(float).fillna(0)

        # Group by ingredient and unit_measure
        ing_df = ing_df.groupby(['ingredient', 'unit_measure'], as_index=False).sum()
        ing_df = ing_df.sort_values(by = 'ingredient')
        ing_df = ing_df.set_index('ingredient')

        # Multiply quantities by number of people
        ing_df['quantity'] = ing_df['quantity'] * self.people

        # Round to nearest 50'
        ing_df['quantity'] = [(x[0]+50 - (x[0] % 50)) if x[0] != 0 and x[1] != 'nan'
            else x[0] for x in zip(ing_df['quantity'], ing_df['unit_measure'])]
        print(ing_df[ing_df['unit_measure'] == 'nan'])

        # Convert to str and replace 0 values with empty string
        ing_df['quantity'] = ing_df['quantity'].astype('int').astype('str')
        ing_df['quantity'] = ing_df['quantity'].replace('0', ' ')

        # Replace nas in unite unit measure with empty string
        ing_df['unit_measure'] = ing_df['unit_measure'].astype('str')
        ing_df['unit_measure'] = ing_df['unit_measure'].replace('nan', ' ')

        # Create ingredients list
        shopping_list = ing_df.transpose().to_dict()

        return shopping_list